In [1]:
import agents_env
from agents_env.agent_replay_motion import HumanoidReplay
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *
from copy import deepcopy

In [2]:

from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random

Here I will have my agent

In [199]:
from agents_env.agent_env_template import HumanoidDiff
from some_math.rotation6D import quaternion_to_rotation_6d

In [4]:
class HumanoidEnvTrain(HumanoidDiff):
    def __init__(self, reference_trajectory_qpos, 
                 reference_trajectory_qvel, 
                 duration_trajectory,
                 dict_duration, 
                 model_path,
                 kp_gains,
                 kd_gains,
                 reference_x_pos,
                 reference_x_rot,
                 **kwargs):
        super().__init__(reference_trajectory_qpos, 
                         reference_trajectory_qvel, 
                         duration_trajectory, 
                         dict_duration, 
                         model_path, **kwargs)
        self.kp__gains = kp_gains
        self.kd__gains = kd_gains
        #the row lenght is the number of frames of the motion
        #thus it is the lenght, it will be the same with qvel
        self.rollout_lenght = reference_trajectory_qpos.shape[0]
        #I want to save another instance of the model for the refernece 
        self.sys_reference = deepcopy(self.sys)
        self.reference_x_pos = reference_x_pos
        self.reference_x_rot = reference_x_rot
        #save the joints idx
        self.set_joints_one_idx()
        
    #here I will set the joints indexes 
    #the joints that only have one dofs
    #the last paremeter X doesnt matter it doesnt mean anything
    #the actuator idx is depending on the nu dim, so is the qpos
    #after getting rid of the first 7 values
    def set_joints_one_idx(self):
        
        self.right_elbow_joint = get_actuator_indx(self.sys.mj_model,'right_elbow','X')
        self.left_elbow_joint = get_actuator_indx(self.sys.mj_model,'left_elbow','X')
        self.right_knee_joint = get_actuator_indx(self.sys.mj_model,'right_knee','X')
        self.left_knee_joint = get_actuator_indx(self.sys.mj_model,'left_knee','X')
    
    
    #set pd callback
    def set_pd_callback(self,pd_control):
        self.pd_function = pd_control
 
    
    
    def _demo_replay(self, state)-> State:
        pass
    
    
    def reset(self, rng: jp.ndarray) -> State:
        
        #set this as zero
        reward, done, zero = jp.zeros(3)
        #random state initialization (RSI)
        new_step_idx = jax.random.randint(rng, shape=(), minval=0, maxval=self.rollout_lenght)
        #data = self.get_reference_state(new_step_idx)
        data = self.get_reference_state(1)
        metrics = {'step_index': new_step_idx, 'pose_error': zero, 'fall': zero}
        obs = self._get_obs(data, new_step_idx)
        
        state = State(data, obs, reward, done, metrics)
        
        #update the replay with 0 index
        state.metrics.update(replay=jp.zeros(1)[0])
        
        return state
    
    def _get_obs(self, data: mjx.Data, step_idx: jp.ndarray)-> jp.ndarray:
          
          current_step_inx =  jp.asarray(step_idx, dtype=jp.int32)
          #we take out the first index that is the world pos
          #we dont care about the world pos, we care starting from the root
          #this should be xpos
          current_xpos = data.x.pos
          
          relative_pos = (current_xpos - current_xpos[0])[1:].ravel()
          
          
          current_qpos_root = data.qpos[3:7]
          #qpos of the joins this are scale values, since they are hinge joints
          current_qpos_joints = data.qpos[7:] 
          #now I will convert them into quaterions
          #first the joints that are onedofs, thus axis angle to quaterion
          #current_orientations
        
          
          #in theory it is mutiable if we do concatenate [] instead of
          #() since, one is a list and the other a tuple
          return jp.concatenate([relative_pos,data.qpos,data.qvel])
   
    
    
    #this will grab a reference state from the reference trajector
    #dont confuse with the state of the main agent
    def set_new_ref_state(self,step_index):
        #grab the current qpos and qvel of the reference
        
        ref_qp = self.reference_trajectory_qpos[step_index]
        ref_qv = self.reference_trajectory_qvel[step_index]
        
        data = mjx.make_data(self.sys_reference)
        data = data.replace(qpos=ref_qp, qvel=ref_qv)
        
        self.new_ref_data = mjx.forward(self.sys_reference, data)
        
    
    
    def get_com_reference(self):
        #here I want to calculate the center of mass
        #we only want the root
        return self.new_ref_data.subtree_com[1]
        
    
    def get_reference_state(self,step_index):
        ref_qp = self.reference_trajectory_qpos[step_index]
        ref_qv = self.reference_trajectory_qvel[step_index]
        #now I will return a state depending on the index and the reference trajectory
        return self.pipeline_init(ref_qp,ref_qv)
        
    #just with a custom target but not selected joints
    def step(self, state: State, action: jp.ndarray,
                                           custom_target,time) -> State:
        #note this is how it looks a state
        """
        @struct.dataclass
        class State(base.Base):
            #Environment state for training and inference
            pipeline_state: Optional[base.State]
            obs: jax.Array
            reward: jax.Array
            done: jax.Array
            metrics: Dict[str, jax.Array] = struct.field(default_factory=dict)
            info: Dict[str, Any] = struct.field(default_factory=dict)

        """              
        
        initial_idx = state.metrics['step_index']
        current_step_inx =  jp.asarray(initial_idx, dtype=jp.int32)
       
        self.set_new_ref_state(current_step_inx)
        #jax.debug.print("step: {}",current_step_inx)
        com_ref = self.get_com_reference()
        #jax.debug.print("com: {}",com_ref)
        xpos = self.reference_x_pos[current_step_inx]
        #jax.debug.print("xpos: {}",xpos)
        
        xrot = self.reference_x_rot[current_step_inx]
        #jax.debug.print("xrot: {}",xrot)
        
        
        
        #increment the step index to know in which episode we are
        next_step_index = state.metrics['step_index'] +1
        #jax.debug.print("step: {}",step_index)  
        #time_debugger = state.pipeline_state.time
        #jax.debug.print("time: {}",time_debugger)  
        
        #current qpos and qvel for the torque    
        qpos = state.pipeline_state.q
        qvel = state.pipeline_state.qd
    
        torque = self.pd_function(custom_target,self.sys,state,qpos,qvel,
                                 self.kp__gains,self.kd__gains,time,self.sys.dt) 
        
        data = self.pipeline_step(state.pipeline_state,torque)
      
        
        done =1.0
        reward = jp.zeros(3)
        
        obs = self._get_obs(data, next_step_index)
        
        
        state.metrics.update(
            step_index=next_step_index,
            pose_error=jp.zeros(3),
            fall=jp.zeros(3),
        )
        
        
        return state.replace(
            pipeline_state=data, obs=obs, reward=reward, done=done
        )
        
        
        
    

In [5]:
#since we are going to use custom trajectory, we will set up the initial
#and the end time
t_init = 1
t_end = 3

In [6]:
#this is just dummy data to initialize the agent
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid_no_gravity.xml'

data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

data_xpos_mocap=jp.asarray(trajectory.data_xpos)
data_xrot_mocap=jp.asarray(trajectory.data_xrot)


In [7]:
#get th kp and kd for the agent
kp,kd = generate_kp_kd_gains()
print(kp)
print(kd)

[1000 1000 1000  100  100  100  400  400  400  300  400  400  400  300
  500  500  500  500  400  400  400  500  500  500  500  400  400  400]
[100 100 100  10  10  10  40  40  40  30  40  40  40  30  50  50  50  50
  40  40  40  50  50  50  50  40  40  40]


In [8]:
envs.register_environment('humanoidEnvMimic', HumanoidEnvTrain)
env_name = 'humanoidEnvMimic'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path,
                           kp_gains = kp,
                           kd_gains = kd,
                           reference_x_pos=data_xpos_mocap,
                           reference_x_rot=data_xrot_mocap)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)


#select the pd_control
from agents_env.pds_controllers_agents import stable_pd_controller_custom_trajectory
env.set_pd_callback(stable_pd_controller_custom_trajectory)

In [9]:
from some_math.math_utils import generate_trajectory,compute_cubic_trajectory,start_trajectories

In [10]:
trajec_dict = dict()


a_jnt_right =get_actuator_indx(env.sys.mj_model,'right_shoulder','Y')
a_jnt_left =get_actuator_indx(env.sys.mj_model,'left_shoulder','X')
a_jnt_left_elbow = get_actuator_indx(env.sys.mj_model,'left_elbow','X')
a_jnt_right_elbow = get_actuator_indx(env.sys.mj_model,'right_elbow','X')

right_knee = get_actuator_indx(env.sys.mj_model,'right_knee','X')
left_knee = get_actuator_indx(env.sys.mj_model,'left_knee','X')

trajec_dict[a_jnt_right] = generate_trajectory(t_init,t_end, 0, -1.5)
#trajectory left
trajec_dict[a_jnt_left] = generate_trajectory(t_init, t_end, 0, 1.5)
# #left elbow   
trajec_dict[a_jnt_left_elbow]= generate_trajectory(t_init, t_end, 0, 1.5)
# #right elbow
trajec_dict[a_jnt_right_elbow] = generate_trajectory(t_init, t_end, 0, 1.5)

trajec_dict[right_knee] = generate_trajectory(t_init, t_end, 0, 0)

trajec_dict[left_knee] = generate_trajectory(t_init, t_end, 0, 0)


start_trajec = start_trajectories(trajec_dict)

In [11]:
print(a_jnt_right)
print(a_jnt_left)
print(a_jnt_left_elbow)
print(a_jnt_right_elbow)
print(right_knee)
print(left_knee)


7
10
13
9
17
24


In [12]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500

In [89]:

for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    time = jp.clip(time, t_init, t_end)
    
   
         
    state = jit_step(state, ctrl,start_trajec,time)
    
    rollout.append(state.pipeline_state)

IndexError: Too many indices for array: 2 non-None/Ellipsis indices for dim 1.

In [ ]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.


In [100]:
state.pipeline_state.xpos

Array([[0.        , 0.        , 0.        ],
       [1.0449276 , 0.9201753 , 0.762677  ],
       [1.0468391 , 0.9241122 , 0.99878746],
       [1.0308832 , 1.004719  , 1.2070577 ],
       [1.2122096 , 1.0383474 , 1.229011  ],
       [1.4756391 , 1.1030678 , 1.272881  ],
       [0.8589965 , 0.9421486 , 1.2391825 ],
       [0.73543   , 0.7370127 , 1.1044271 ],
       [1.1253372 , 0.9473576 , 0.7615728 ],
       [1.2434226 , 1.2149004 , 0.45796478],
       [1.2533716 , 1.0272739 , 0.09369731],
       [0.96451795, 0.89299303, 0.76378125],
       [0.75786364, 0.86453366, 0.39746842],
       [0.75479364, 0.6369034 , 0.05663341]], dtype=float32)

In [138]:
state.pipeline_state.x.rot

Array([[-0.9988232 , -0.0112808 , -0.04714939, -0.00136157],
       [-0.9760239 ,  0.00921053, -0.21670713,  0.01817523],
       [-0.954311  ,  0.05125172, -0.01494027, -0.29400754],
       [-0.7900792 ,  0.11564598, -0.52131987, -0.30104205],
       [-0.90522087, -0.15832108,  0.27672157, -0.2809533 ],
       [-0.9292731 , -0.30408332, -0.00979375,  0.20949635],
       [-0.8230528 , -0.16857538,  0.43154195,  0.32853878],
       [-0.9441449 ,  0.07370024,  0.21859391,  0.23531947],
       [-0.89839023, -0.07485342, -0.36343905,  0.23495519],
       [-0.90135527,  0.1235581 , -0.23688422,  0.34084862],
       [-0.90527534, -0.10094515,  0.3854131 , -0.14745612],
       [-0.97898144, -0.01743659, -0.09829593, -0.177846  ],
       [-0.97810173, -0.02084978, -0.10072432, -0.18093319]],      dtype=float32)

Testing how to get the global rotations

In [176]:


# Remove the first character ('f') and convert the rest to a list of integers
link_types_numbers = [int(char) for char in env.sys.link_types[1:]]

# Convert the list of integers to a JAX array
link_types_array_without_root = jp.array(link_types_numbers)


In [107]:
env.sys.jnt_axis

Array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]], dtype=float32)

In [31]:
#save the root qpos
current_pos_rot = state.pipeline_state.qpos[0:3]
current_root_rot = state.pipeline_state.qpos[3:7]
current_joints = state.pipeline_state.qpos[7:]

print(state.pipeline_state.qpos)



[ 6.97175786e-03  5.95309353e-03  7.52526999e-01 -9.98823225e-01
 -1.12807984e-02 -4.71493937e-02 -1.36156718e-03 -3.17845419e-02
  3.42894375e-01 -3.98943871e-02 -8.93776119e-02 -4.40016717e-01
  5.91198862e-01 -6.19465292e-01  5.02453387e-01  7.96520531e-01
  1.75985932e+00  7.12900817e-01 -4.34377730e-01 -1.01669244e-01
  9.86902952e-01 -3.00477624e-01 -4.77567464e-01 -5.72591901e-01
 -1.22387028e+00 -2.82287776e-01 -2.97072321e-01 -3.64673197e-01
  4.22074705e-01 -8.18987012e-01  5.31627297e-01 -1.00513864e+00
  6.95715472e-03  3.84540367e-03  6.93053333e-03]


In [32]:
#now get the indices for the one joint index


joints_one_dofs_indices=jp.array([env.right_elbow_joint,env.left_elbow_joint,env.right_knee_joint,env.left_knee_joint])
print(joints_one_dofs_indices)


[ 9 13 17 24]


In [35]:
def axis_angle_to_quat(axis: jax.Array, angle: jax.Array) -> jax.Array:
  """Provides a quaternion that describes rotating around axis by angle.

  Args:
    axis: (3,) axis (x,y,z)
    angle: () float angle to rotate by

  Returns:
    A quaternion that rotates around axis by angle
  """
  s, c = jp.sin(angle * 0.5), jp.cos(angle * 0.5)
  return jp.insert(axis * s, 0, c)


In [36]:
def quat_mul(u: jax.Array, v: jax.Array) -> jax.Array:
  """Multiplies two quaternions.

  Args:
    u: (4,) quaternion (w,x,y,z)
    v: (4,) quaternion (w,x,y,z)

  Returns:
    A quaternion u * v.
  """
  return jp.array([
      u[0] * v[0] - u[1] * v[1] - u[2] * v[2] - u[3] * v[3],
      u[0] * v[1] + u[1] * v[0] + u[2] * v[3] - u[3] * v[2],
      u[0] * v[2] - u[1] * v[3] + u[2] * v[0] + u[3] * v[1],
      u[0] * v[3] + u[1] * v[2] - u[2] * v[1] + u[3] * v[0],
  ])


In [40]:
#this works properly
right_elbow = current_joints[9]
right_elbow_axis = env.sys.jnt_axis[10]

right_elbow_qloc = axis_angle_to_quat(right_elbow_axis,right_elbow )

quat = quat_mul(state.pipeline_state.x.rot[3], right_elbow_qloc)


quat




Array([-0.90522087, -0.15832108,  0.27672157, -0.2809533 ], dtype=float32)

In [113]:
def compute_quaternion_for_joint(joint_axis,joint_angle):
    
    joint_quat = axis_angle_to_quat(joint_axis,joint_angle)
    return joint_quat

In [150]:

def combine_quaterions_joint_3DOFS(quats):
    # Combine rotations in XYZ order, ensure the multiplication reflects this
    #first element is w the real number
    combined_quat = quat_mul(quat_mul(quats[0], quats[1]),quats[2])
    return combined_quat


In [235]:
#now do the same but with vmap
#jnt_axis without free joint
axis_hinge = env.sys.jnt_axis[1:]

vmap_compute_quaternion = jax.vmap(compute_quaternion_for_joint, in_axes=(0, 0))

hinge_quat = vmap_compute_quaternion(axis_hinge,current_joints) 

print(hinge_quat)
print(hinge_quat.shape)

[[ 0.9998737  -0.0158916  -0.         -0.        ]
 [ 0.98533887  0.          0.17060849  0.        ]
 [ 0.99980104 -0.         -0.         -0.01994587]
 [ 0.9990016  -0.04467393 -0.         -0.        ]
 [ 0.97589564 -0.         -0.21823777 -0.        ]
 [ 0.95662767  0.          0.          0.29131332]
 [ 0.9524151  -0.30480403 -0.         -0.        ]
 [ 0.9686082   0.          0.24859233  0.        ]
 [ 0.9217371   0.          0.          0.38781536]
 [ 0.63720536  0.         -0.7706941   0.        ]
 [ 0.93714136  0.34894997  0.          0.        ]
 [ 0.97650707 -0.         -0.21548538 -0.        ]
 [ 0.9987082  -0.         -0.         -0.05081273]
 [ 0.8807033   0.         -0.47366843  0.        ]
 [ 0.9887354  -0.14967425 -0.         -0.        ]
 [ 0.97162634 -0.         -0.23652105 -0.        ]
 [ 0.95929646 -0.         -0.         -0.28240088]
 [ 0.8185379  -0.          0.5744525  -0.        ]
 [ 0.99005574 -0.14067572 -0.         -0.        ]
 [ 0.98898876 -0.         -0.14

In [234]:
# Mask to filter out the quaternions that are not to be combined in triples
mask = jp.ones(hinge_quat.shape[0], dtype=bool)
mask = mask.at[joints_one_dofs_indices].set(False)



#index 0 since is a tuple and we want the indices that are index 0
combinable_indices = jp.where(mask)[0]
non_combinable_indices = jp.where(~mask)[0]

no_grouped_quaterions = hinge_quat[non_combinable_indices]


#select and store these indices
# Assuming the remaining quaternions are multiple of three
# Reshape the array to (-1, 3, 4) where 3 is the number of quaternions to be combined and 4 is the quaternion dimension
grouped_quaternions = hinge_quat[combinable_indices].reshape(-1, 3, 4)
#this will be applied on the first axs
vmap_combined_quat = jax.vmap(combine_quaterions_joint_3DOFS)

quat_combined = vmap_combined_quat(grouped_quaternions)

#there are 13 links -12, since we will merge the root at the end the shape 1 is 4 for the quat
quat_loc_all_joints = jp.zeros((state.pipeline_state.x.rot.shape[0]-1,state.pipeline_state.x.rot.shape[1]))

# Create a mask where each position is True if the corresponding link type is 3
link_types_mask = link_types_array_without_root == 3

filter_out_jnt_type_idx = jp.where(link_types_mask)

filter_out_jnt_type_one_dofs = jp.where(~link_types_mask)

quat_loc_all_joints = quat_loc_all_joints.at[filter_out_jnt_type_idx].set(quat_combined)

quat_loc_all_joints = quat_loc_all_joints.at[filter_out_jnt_type_one_dofs].set(no_grouped_quaterions)

quat_loc_all_joints = jp.concatenate([current_root_rot.reshape(1,-1),quat_loc_all_joints],axis=0)


print(quat_loc_all_joints)

# #testing it it works the global chest

# global_quat_chest = quat_mul(state.pipeline_state.x.rot[0],quat_loc_all_joints[1])

# print(global_quat_chest)
#now to 6D rotatio
sixD_matrix = quaternion_to_rotation_6d(quat_loc_all_joints)

print(sixD_matrix)
print(sixD_matrix.shape)

[[-0.9988232  -0.0112808  -0.04714939 -0.00136157]
 [ 0.9849643  -0.019058    0.17024067 -0.02236166]
 [ 0.9297966  -0.10521828 -0.19586344  0.29333425]
 [ 0.8797037  -0.18030933  0.33273026  0.28792447]
 [ 0.63720536  0.         -0.7706941   0.        ]
 [ 0.91012216  0.3505731  -0.18436486 -0.12159649]
 [ 0.8807033   0.         -0.47366843  0.        ]
 [ 0.9315755  -0.0734667  -0.26540676 -0.23733708]
 [ 0.8185379  -0.          0.5744525  -0.        ]
 [ 0.9666973  -0.11025239 -0.16931759 -0.15707439]
 [ 0.88737756  0.0831369  -0.4261168   0.15515728]
 [ 0.8763479  -0.          0.48167875 -0.        ]
 [ 0.99998605  0.00348521  0.00191062  0.00347192]]
[[ 0.99555016 -0.00165616  0.09421854  0.0037837   0.9997418  -0.02240665]
 [ 0.9410361   0.03756199  0.3362144  -0.05053978  0.9982735   0.02992918]
 [ 0.75118506 -0.5042655  -0.42595455  0.5866992   0.80576825  0.08075629]
 [ 0.6127801  -0.6265652   0.48157722  0.38658774  0.76917607  0.50883996]
 [-0.18793869 -0.         -0.9821808

In [91]:

def euler_to_quaternion(x, y, z):
    """
    Convert Euler angles (in radians) to a quaternion using JAX.
    Assumes the rotation sequence is XYZ.

    Parameters:
    x (float): Rotation around the X-axis in radians.
    y (float): Rotation around the Y-axis in radians.
    z (float): Rotation around the Z-axis in radians.

    Returns:
    jnp.ndarray: Quaternion [w, x, y, z] representing the rotation.
    """
    # Precompute sines and cosines of the half angles
    cx = jp.cos(x / 2)
    sx = jp.sin(x / 2)
    cy = jp.cos(y / 2)
    sy = jp.sin(y / 2)
    cz = jp.cos(z / 2)
    sz = jp.sin(z / 2)

    # Compute quaternion components
    w = cx * cy * cz + sx * sy * sz
    x = sx * cy * cz - cx * sy * sz
    y = cx * sy * cz + sx * cy * sz
    z = cx * cy * sz - sx * sy * cz

    return jp.array([w, x, y, z])

In [96]:

#this never work not sure why
from some_math.math_utils import get_global_rotation_quat
from brax import math
#test with the chest and root
#chest qpos
chest_rot = current_joints[0:3]

chest_rot_quat = euler_to_quaternion(chest_rot[0],chest_rot[1],chest_rot[2])

global_chest_rot_quat = get_global_rotation_quat(current_root_rot,chest_rot_quat)

print(current_root_rot)
print(chest_rot)
print(chest_rot_quat)
print(global_chest_rot_quat)

#axis_to_quat_jax(JOINTS_AXIS_ONEDOF['right_elbow'],10)



[-0.9988232  -0.0112808  -0.04714939 -0.00136157]
[-0.03178454  0.34289438 -0.03989439]
[ 0.98507243 -0.01225299  0.17086533 -0.01694026]
[-0.97601832  0.00215754 -0.21728424  0.01307386]


In [148]:
def update_rotation(local_angle, local_axis):
    # Convert the joint angle and axis to a quaternion
    local_rotation = axis_angle_to_quat(local_axis, local_angle)
    # Apply the local rotation to the parent rotation
    return local_rotation  

# Compute local rotations correctly applying local transformation first
chest_x_qloc = update_rotation(current_joints[0], env.sys.jnt_axis[1])
chest_y_qloc = update_rotation(current_joints[1], env.sys.jnt_axis[2])
chest_z_qloc = update_rotation(current_joints[2], env.sys.jnt_axis[3])

# Combine rotations in XYZ order, ensure the multiplication reflects this
combined_quat = quat_mul(quat_mul(chest_x_qloc, chest_y_qloc), chest_z_qloc)


print(combined_quat)


global_quat_chest = quat_mul(state.pipeline_state.x.rot[0],combined_quat)
print(global_quat_chest)

[ 0.9849643  -0.019058    0.17024067 -0.02236166]
[-0.976024    0.00921053 -0.21670713  0.01817523]


Array([-0.976024  ,  0.00921053, -0.21670713,  0.01817523], dtype=float32)